![](./Resources/ai-sk-add-wechat.png)

# Semantic Kernel 自由切换大模型
上一节中，我们介绍了Semantic Kernel如何对接国内智谱AI大模型，但是国内大模型百花齐放，并不是一枝独秀。因此进行应用开发时，系统可能会选择对接多个模型提供商。那如何集成多个大模型并能在自由切换呢？这一节我们来详细展开说说。

从上节可知，**对接国内大模型的主要思路是在OpenAI Connector的基础上来操作的，具体做法就是通过修改`Endpoint`或自定义`OpenAIClient`，但前提是国内大模型的API 要兼容OpenAI的 API。如果不兼容还可尝试代理方案，比如OneApi，查看目标大模型是否已被适配，若适配则直接用代理方案**。也就是说只要大模型的API最终能兼容OpenAI，那对接思路是一样的。

## 注册流行大模型

就目前已知信息而言，通义千问、智谱AI、讯飞星火、月之暗面、零一万物提供了兼容OpenAI的API 格式。请按需访问以下开放平台注册账号并创建API Key。

1. 讯飞星火开放平台：https://www.xfyun.cn
2. 月之暗面开放平台：https://platform.moonshot.cn
3. 零一万物开放平台：https://platform.lingyiwanwu.com
4. 通义千问开放平台：https://dashscope.aliyun.com
5. 智谱清言开放平台：https://open.bigmodel.cn
6. etc.

## 大模型平台抽象
如果直接使用Http对接大模型，至少需要提供以下信息：
1. Uri：请求地址，一个完整的请求地址又包含以下：( 以OpenAI的聊天补全接口为例：`https://api.openai.com/v1/chat/completions` )
    * ApiEndpoint: `https://api.openai.com`
    * ServicePath: `/v1/chat/completions`
2. Model Id：模型编码
3. Api Key：模型密钥

但考虑到市面上有很多AI服务提供商，不同AI服务提供商的对接方式也各有不同，因此官方提供了多个Connectors，比如：
* Microsoft.SemanticKernel.Connectors.OpenAI （√）
* Microsoft.SemanticKernel.Connectors.AzureOpenAI（√）
* Microsoft.SemanticKernel.Connectors.HuggingFace 
* Microsoft.SemanticKernel.Connectors.Google
* Microsoft.SemanticKernel.Connectors.Onnx 
* Microsoft.SemanticKernel.Connectors.MistralAI
* Microsoft.SemanticKernel.Connectors.Ollam 

因此首先需要抽象一个AI 服务提供商类型，对于没有单独Connector的AI 服务提供商，如果其API 格式是兼容OpenAI API 格式的，我们可以将其归类为`OpenAI_Compatible`类型。

In [41]:
public enum AiProviderType
{
    OpenAI,
    OpenAI_Compatible,
    AzureOpenAI,
    HuggingFace,
    Google,
    Onnx,
    MistralAI,
    Ollam
}

紧接着基于以上信息可以抽象以下模型：`AiProvider` 和`ApiService`：

In [42]:
public  class AiProvider
{
    /// <summary>
    /// AI 服务提供商名称
    /// </summary>
    public string Name { get; set; }

    /// <summary>
    /// AI 服务提供商编码
    /// </summary>
    public string Code { get; set; }

    public string ApiKey { get; set; }

    public string ApiEndpoint { get; set; }    
    
    public  AiProviderType AiType { get; set; }

    public List<ApiService> ApiServices { get; set; }

    public ApiService? GetEmbeddingApiService() => GetApiService("embeddings");

    public ApiService? GetChatCompletionApiService() => GetApiService("chat-completions");

    private ApiService? GetApiService(string apiServiceName) => ApiServices.FirstOrDefault(x => x.Name == apiServiceName);
}

/// <summary>
/// AI 服务提供商提供的API 服务
/// </summary>
/// <param name="Name">Api名称</param>
/// <param name="ModelId">模型编码</param>
///  <param name="Models">可用模型列表</param>
public record ApiService(string Name, string ModelId, string[]? ModelIds = null);

但对于一个应用来说，可以包含多个AI 提供商并指定默认的提供商，因此可以进一步抽象一个`AiOptions`：

In [43]:
public class AiOptions
{
    public string DefaultProvider { get; set; }
    public List<AiProvider> Providers { get; set; }

    public AiProvider? GetProvider(string providerCode) =>
        Providers.FirstOrDefault(x => x.Code == providerCode);
}

根据以上抽象，可以添加一个`Config`文件夹然后添加一个`appsettings.json`配置文件，其中配置如下：
```
{
  "AI": {
    "DefaultProvider": "zhipu",
    "Providers": [
      {
        "Name": "智谱",
        "Code": "zhipu",
        "AiType": "OpenAI_Compatible",
        "ApiKey": "ddddc01549175d4c18e65a70e0d8329d.jAFiuW3FfK5PbxuE",
        "ApiEndpoint": "https://open.bigmodel.cn/api/paas/v4",
        "ApiServices": [
          {
            "Name": "chat-completions",
            "ModelId": "glm-4"
          },
          {
            "Name": "embeddings",
            "ModelId": "embedding-3"
          }
        ]
      },{
        "Name": "月之暗面",
        "Code": "moonshot",
        "AiType": "OpenAI_Compatible",
        "ApiKey": "sk-aPvBNM6rz76l7eCcq76f2i2BO3RrvxXITXgYvKT9gEX0y5ek",
        "ApiEndpoint":"https://api.moonshot.cn",
        "ApiServices": [
          {
            "Name": "chat-completions",
            "ModelId": "moonshot-v1-8k"
          }
        ]
      },      
      {
        "Name": "Azure OpenAI",
        "Code": "azure-openai",
        "ApiKey": "163ed6f8bc2947e0906d6ee5e173a222",
        "ApiEndpoint": "https://my-openapi.openai.azure.com",
        "ApiServices": [
          {
            "Name": "chat-completions",
            "ModelId": "gpt-4o"
          },
          {
            "Name": "embeddings",
            "ModelId": "text-embedding-ada-002"
          }
        ]
      },
      {
        "Name": "One Api",
        "Code": "oneapi",
        "AiType": "OpenAI_Compatible",
        "ApiKey": "sk-9y9939P3ufwHaltcB95d91F3D9D64303Ad799e991f4700F1",        
        "ApiEndpoint": "http://localhost:3000/v1",
        "ApiServices": [
          {
            "Name": "chat-completions",
            "ModelId": "lite",
            "ModelIds":["lite","glm-4-flash"]
          }
        ]
      }
  }      
```

从中可以看出共配置了两个AI 提供商：智谱和月之暗面，其中智谱配置了2个API 服务（`chat-completions`和`embedding`），月之暗面仅配置了一个API服务（`chat-completions`）。

有了以上配置，就可以使用[Options Pattern in .NET](https://learn.microsoft.com/en-us/dotnet/core/extensions/options)来加载配置并绑定到对应的`AiOptions`。具体操作如下：

1. 安装以下NuGet包：

In [44]:
#r "nuget:Microsoft.Extensions.Configuration"
#r "nuget:Microsoft.Extensions.Configuration.Json"
#r "nuget:Microsoft.Extensions.Configuration.Binder"
#r "nuget:Microsoft.SemanticKernel"

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Binder, 8.0.2 Microsoft.Extensions.Configuration.Json, 8.0.1 Microsoft.SemanticKernel, 1.25.0

2. 添加工具类`AiSettings`用于加载配置，代码如下所示：

In [45]:
using System.IO;
using Microsoft.Extensions.Configuration;

public static class AiSettings
{
    public static AiOptions LoadAiProvidersFromFile()
    {
        var configuration = new ConfigurationBuilder()
            .SetBasePath(Directory.GetCurrentDirectory())
            .AddJsonFile("Config/appsettings.json", optional: true, reloadOnChange: true)
            .Build();

        var aiOptions = configuration.GetSection("AI").Get<AiOptions>();
        return aiOptions;
    }
}

In [46]:
// 测试配置读取
var aiOptions= AiSettings.LoadAiProvidersFromFile();
aiOptions.Display();

Submission#39+AiOptions DefaultProvider zhipu Providers index value 0 Submission#38+AiProvider Name 智谱 Code zhipu ApiKey ddddc01549175d4c18e65a70e0d8329d.jAFiuW3FfK5PbxuE ApiEndpoint https://open.bigmodel.cn/api/paas/v4 AiType OpenAI_Compatible ApiServices index value 0 ApiService { Name = chat-completions, ModelId = glm-4, ModelIds = } Name chat-completions ModelId glm-4 ModelIds <null> 1 ApiService { Name = embeddings, ModelId = embedding-3, ModelIds = } Name embeddings ModelId embedding-3 ModelIds <null> 1 Submission#38+AiProvider Name 月之暗面 Code moonshot ApiKey sk-aPvBNM6rz76l7eCcq76f2i2BO3RrvxXITXgYvKT9gEX0y5ek ApiEndpoint https://api.moonshot.cn/v1 AiType OpenAI_Compatible ApiServices index value 0 ApiService { Name = chat-completions, ModelId = moonshot-v1-8k, ModelIds = } Name chat-completions ModelId moonshot-v1-8k ModelIds <null> 2 Submission#38+AiProvider Name Azure OpenAI Code azure-openai ApiKey 163ed6f8bc2947e0906d6ee5e173a222 ApiEndpoint https://my-openapi.openai.azure.com AiType AzureOpenAI ApiServices index value 0 ApiService { Name = chat-completions, ModelId = gpt-4o, ModelIds = } Name chat-completions ModelId gpt-4o ModelIds <null> 1 ApiService { Name = embeddings, ModelId = text-embedding-ada-002, ModelIds = } Name embeddings ModelId text-embedding-ada-002 ModelIds <null> 3 Submission#38+AiProvider Name One Api Code oneapi ApiKey sk-9y9939P3ufwHaltcB95d91F3D9D64303Ad799e991f4700F1 ApiEndpoint http://localhost:3000/v1 AiType OpenAI_Compatible ApiServices index value 0 ApiService { Name = chat-completions, ModelId = lite, ModelIds = System.String[] } Name chat-completions ModelId lite ModelIds [ lite, glm-4-flash, lite, glm-4-flash ]

3. 添加扩展方法来注册聊天补全服务：

In [49]:
using Microsoft.SemanticKernel;
using System.ClientModel;
using OpenAI;

public static IKernelBuilder AddChatCompletionService(this IKernelBuilder builder, string aiProviderCode = null)
{
    var aiOptions = AiSettings.LoadAiProvidersFromFile();
    if (string.IsNullOrWhiteSpace(aiProviderCode))
    {
        aiProviderCode = aiOptions.DefaultProvider;
    }
    var aiProvider = aiOptions.Providers.FirstOrDefault(x => x.Code == aiProviderCode);
    if (aiProvider == null)
    {
        throw new ArgumentException($"未找到编码为 {aiProviderCode} 的 AI 服务提供商");
    }
    var modelId = aiProvider.GetChatCompletionApiService()?.ModelId;
    if (string.IsNullOrWhiteSpace(modelId))
    {
        throw new ArgumentException($"未找到名称为 chat-completions 的 API 服务");
    }    

    if (aiProvider.AiType == AiProviderType.OpenAI)
    {
        builder.AddOpenAIChatCompletion(
            modelId: modelId,
            apiKey: aiProvider.ApiKey);
    }
    
    if (aiProvider.AiType == AiProviderType.AzureOpenAI)
    {
        builder.AddAzureOpenAIChatCompletion(
            deploymentName: modelId,
            endpoint: aiProvider.ApiEndpoint,
            apiKey: aiProvider.ApiKey);
    }

    if (aiProvider.AiType == AiProviderType.OpenAI_Compatible)
    {
        OpenAIClientOptions clientOptions = new OpenAIClientOptions
        {
            Endpoint = new Uri(aiProvider.ApiEndpoint)
        };

        OpenAIClient client = new(new ApiKeyCredential(aiProvider.ApiKey), clientOptions);

        builder.AddOpenAIChatCompletion(modelId: modelId, openAIClient: client);
    }
    
    return builder;
}

5. 注册AI 服务并测试：

In [52]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

// 引入交互式的内核命名空间，以便用户输入
using PolyglotKernel= Microsoft.DotNet.Interactive.Kernel;

var aiProviderCode = await PolyglotKernel.GetInputAsync("请输入AI服务提供商编码：");

//Create Kernel builder
var builder = Kernel.CreateBuilder();

builder.AddChatCompletionService(aiProviderCode);

var kernel = builder.Build();

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();
var response = await chatCompletionService.GetChatMessageContentAsync("你是谁？");
Console.WriteLine(response.Content);

我是一个AI助手，旨在帮助回答问题、提供信息和协助解决问题。如果你有任何问题或需要帮助，请随时告诉我！
